<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Predicting shots made per game by Kobe Bryant

_Instructor: Aymeric Flaisler_

---

In this lab you'll be using regularized regression penalties Ridge, Lasso, and Elastic Net to try and predict how many shots Kobe Bryant made per game in his career.

The Kobe shots dataset has hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model considering the limited number of observations (games) we have available. Furthermore, many of the predictors have significant multicollinearity. 

**Warning:** Some of these calculations are computationally expensive and may take a while to execute.  It may be worth while to only use a portion of the data to perform these calculations, especially if you have experienced kernel issues in the past.

---

### 1. Load packages and data

In [36]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import r2_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [37]:
kobe = pd.read_csv('./datasets/kobe_superwide_games.csv')

---

### 2. Examine the data

- How many columns are there?
- Examine what the observations (rows) and columns represent.
- Why might regularization be particularly useful for this dataset?

In [38]:
# A:
kobe.shape
#645 columns that represent kobe stats in his career



(1558, 645)

In [39]:
# kobe.columns

In [40]:
# with pd.option_context('display.max_rows', 999,'display.max_columns', 999):
#     display(kobe)

In [41]:
kobe.corr().head()

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
SHOTS_MADE,1.000000,0.009671,-0.076029,-0.024370,0.004683,0.004683,0.011791,0.006625,0.011986,-0.017227,...,-0.014039,-0.036108,0.062609,-0.017227,0.006544,0.000266,-0.016149,0.146655,0.059012,0.194842
AWAY_GAME,0.009671,1.000000,-0.000046,-0.000046,0.025310,0.025310,-0.000046,-0.000046,-0.025375,-0.025375,...,-0.000046,-0.021380,0.011718,0.025310,0.014764,0.002704,0.024190,0.053858,0.022408,0.007128
SEASON_OPPONENT:atl:1996-97,-0.076029,-0.000046,1.000000,-0.001285,-0.000909,-0.000909,-0.001285,-0.001285,-0.000909,-0.000909,...,-0.001285,-0.010103,-0.006594,-0.000909,-0.012282,-0.001282,-0.003170,-0.023060,-0.024709,-0.060220
SEASON_OPPONENT:atl:1997-98,-0.024370,-0.000046,-0.001285,1.000000,-0.000909,-0.000909,-0.001285,-0.001285,-0.000909,-0.000909,...,-0.001285,-0.010103,-0.006594,-0.000909,-0.012282,-0.001282,-0.003170,-0.023060,-0.024709,-0.054241
SEASON_OPPONENT:atl:1999-00,0.004683,0.025310,-0.000909,-0.000909,1.000000,-0.000642,-0.000909,-0.000909,-0.000642,-0.000642,...,-0.000909,-0.007142,-0.004661,-0.000642,-0.008682,-0.000906,-0.002241,-0.016301,-0.033038,-0.030961


---

### 3. Make predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

In [42]:
# A:
from sklearn.preprocessing import StandardScaler
st= StandardScaler()
x = kobe.drop('SHOTS_MADE',axis = 1)
y= kobe['SHOTS_MADE']


---

### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?

In [43]:
# A:
x_std= st.fit_transform(x)
lr = LinearRegression()
lr.fit(x,y)
lr.score(x,y)


C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


0.8078936863518436

---

### 5. Find an optimal value for Ridge regression alpha using `RidgeCV`.

[Go to the documentation and read how RidgeCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)

> *Hint: once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that Ridge performs best searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit!


In [44]:
# A:
rid = RidgeCV(alphas=np.logspace(0, 1, 20.0),cv=5)
rid.fit(x_std,y)

C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RidgeCV(alphas=array([ 1.     ,  1.12884,  1.27427,  1.43845,  1.62378,  1.83298,
        2.06914,  2.33572,  2.63665,  2.97635,  3.35982,  3.79269,
        4.28133,  4.83293,  5.45559,  6.15848,  6.95193,  7.8476 ,
        8.85867, 10.     ]),
    cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [29]:
# rid.coef_

In [30]:
# np.logspace

In [45]:
rid.alpha_

10.0

---

### 6. Cross-validate the Ridge regression $R^2$ with the optimal alpha.

Is it better than the Linear regression? If so, why might this be?

In [46]:
# A:
rid_score= cross_val_score(rid,x_std,y,cv=5)
rid_score

C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


array([0.66973164, 0.51534941, 0.49020935, 0.36034155, 0.33683081])

In [47]:
np.mean(rid_score)

0.47449255305131854

---

### 7. Find an optimal value for Lasso regression alpha using `LassoCV`.

[Go to the documentation and read how LassoCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) It is very similar to `RidgeCV`.

> *Hint: again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that Lasso, unlike Ridge, performs best searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide itself what alphas to use by instead setting the keyword argument `n_alphas=` to however many alphas you want it to search over. It is recommended to let sklearn choose the range of alphas.

_**Tip:** If you find your CV taking a long time and you're not sure if its working set `verbose =1`._

In [49]:
# A:
las = LassoCV()
las.fit(x_std,y)
las.score(x_std,y)

C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.6957998358025518

In [51]:
a=las.alpha_
a

0.08015435874757074

---

### 8. Cross-validate the Lasso $R^2$ with the optimal alpha.

Is it better than the Linear regression? Is it better than Ridge? What do the differences in results imply about the issues with the dataset?

In [53]:
# A:
lass= Lasso(alpha=a)
las_score=cross_val_predict(lass,x_std,y,cv=5)


In [54]:
las_score

array([ 0.20991628,  0.12952211,  0.85480827, ...,  8.11799552,
        5.63124015, 17.6126715 ])

In [55]:
np.mean(las_score)

7.363162095070048

---

### 9. Look at the coefficients for variables in the Lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original dataset are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** if you only fit the Lasso within `cross_val_score`, you will have to refit it outside of that
function to pull out the coefficients.

In [ ]:
kobe['f_im']=kobe.coe

In [61]:
# # A:
def lasso_coefs(X, Y, alphas):
    coefs = []
    lasso_reg = Lasso()
    for a in alphas:
        lasso_reg.set_params(alpha=a)
        lasso_reg.fit(X, Y)
        coefs.append(lasso_reg.coef_)
        
    return coefs


In [67]:
l_alphas = np.arange(0.001, 0.15, 0.0025)
l_coefs = lasso_coefs(x_std,y, l_alphas)
kobe['coeff']=l_coefs

C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\mgood\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increa

ValueError: Length of values does not match length of index

In [66]:
kobe.sort_values

<bound method DataFrame.sort_values of       SHOTS_MADE  AWAY_GAME  SEASON_OPPONENT:atl:1996-97  \
0            0.0          0                          0.0   
1            0.0          1                          0.0   
2            2.0          1                          0.0   
3            2.0          1                          0.0   
4            0.0          0                          1.0   
5            1.0          1                          0.0   
6            2.0          1                          0.0   
7            1.0          0                          0.0   
8            4.0          1                          0.0   
9            0.0          1                          0.0   
10           2.0          0                          0.0   
11           4.0          1                          0.0   
12           1.0          1                          0.0   
13           0.0          1                          0.0   
14           1.0          0                          0.0   
1

---

### 10. Find an optimal value for Elastic Net regression alpha using `ElasticNetCV`.

[Go to the documentation and read how ElasticNetCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note here that you will be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: strength of regularization
- `l1_ratio`: amount of ridge vs. lasso (0 = all ridge, 1 = all lasso)
    
Do not include 0 in the search for `l1_ratio`: it will not allow it and break!

You can use `n_alphas` for the alpha parameters instead of setting your own values: highly recommended!

Also - be careful setting too many l1_ratios over cross-validation folds in your search. It can take a very long time if you choose too many combinations and for the most part there are diminishing returns in this data.

In [70]:
# A:
optE = ElasticNetCV(l1_ratio=0.5, n_alphas=300, cv=10, verbose=1)
optE.fit (x_std,y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=1000, n_alphas=300, n_jobs=None,
       normalize=False, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=1)

In [75]:
optE.alpha_

0.12152262715421397

In [73]:
optE.l1_ratio

0.5

---

### 11. Cross-validate the ElasticNet $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the Ridge and Lasso regularized regressions?

In [77]:
# A:
Ent = ElasticNet(alpha=optE.alpha_,l1_ratio=optE.l1_ratio)
Escore= cross_val_score(Ent,x_std,y,cv=10)


In [78]:
Escore

array([0.61648894, 0.52900196, 0.5321266 , 0.61407517, 0.54465249,
       0.55418064, 0.52689571, 0.44367387, 0.46038708, 0.50237335])

In [79]:
np.mean(Escore)

0.5323855807847229

---

### 12. [Bonus] Compare the residuals for the Ridge and Lasso visually.


In [13]:
# A: Maybe a histogram?